In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from transformers.modeling_outputs import SequenceClassifierOutput

class CustomSentimentClassifierWithWeights(nn.Module):
    def __init__(self, base_model, class_weights, num_labels=3, dropout_rate=0.1):
        super().__init__()
        self.num_labels = num_labels
        self.base_model = base_model
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(768, num_labels)
       
        self.loss_fct = nn.CrossEntropyLoss(weight=class_weights)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0]
        dropped_output = self.dropout(cls_output)
        logits = self.classifier(dropped_output)

        loss = None
        
        if labels is not None:
            loss = self.loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
def load_model_and_tokenizer(model_path="article_sentiment_analysis.pth", base_checkpoint="distilbert-base-uncased"):

    print("Loading sentiment model and tokenizer:")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    print(f"Using device: {device}")

    tokenizer = AutoTokenizer.from_pretrained(base_checkpoint)

    base_model = AutoModel.from_pretrained(base_checkpoint).to(device)

    dummy_weights = torch.ones(3).to(device) 
    
    model = CustomSentimentClassifierWithWeights(
        base_model=base_model,
        class_weights=dummy_weights,
        num_labels=3
    )

    try:
        model.load_state_dict(torch.load(model_path, map_location=device))
    except FileNotFoundError:
        print(f"ERROR: Model file not found at {model_path}")
        print("Please make sure the file is in the same directory.")
        return None, None, None
    except Exception as e:
        print(f"Error loading model state_dict: {e}")
        return None, None, None

    model.to(device)
    model.eval() 
    
    print(" Custom sentiment model loaded.")
    return model, tokenizer, device

In [ ]:
def get_sentiment(text, model, tokenizer, device):
    """
    Gets a single sentiment score [-1, 1] for a given text
    using the loaded model.
    """
    if not isinstance(text, str):
        return 0.0

    inputs = tokenizer(
        text, 
        padding="max_length", 
        truncation=True, 
        max_length=128, 
        return_tensors="pt" 
    )
    
   
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    
    with torch.no_grad(): 
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    probs = torch.softmax(logits, dim=1).cpu().numpy()[0]

    score = probs[2] - probs[0]
    
    return float(score)